In [20]:
import spacy
import urllib.request
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import re
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
# Load only once
try:
    if(nlp):
        pass
except:
    nlp = spacy.load("en_core_web_md")

# Load only once
try:
    if(sr[0]):
        pass
except:
    sr = stopwords.words('english')


def tokenize(text):
    t0 = []
    # Split the text into words separated by any of these chars
    p = re.compile('[-_\", {};:=?\[\]\(\)\'.]')
    
    # Get the word list into an array, t
    t = p.split(text)
    lt = len(t)

    # Add the 4+ chars words into an array, t1. Discard the smaller words
    for i in range(lt):
        w = t[i]
        lw = len(w)
        if(lw > 3):
            t0.append(w)
    return t0

def Lemmatise(text):
# Implementing lemmatization
    p = re.compile('[a-zA-Z]')
    lt = []
    lem = nlp(text)
    # finding lemma for each word
    for word in lem:
#        print(word.text, word.lemma_)
        if(p.findall(str(word))):
            lemma = word.lemma_.lower() 
            if(len(lemma) > 3):
                if str(lemma) not in (sr):
                    lt.append(lemma)
#    print(t1)
    return lt
def CountWords(t):
    global counts
    from collections import Counter
    counts = Counter(t)
#    print(counts)

def ReadUrls(i, j):
    url1 = urls[i]
    url2 = urls[j]
#    print(urls[i], urls[j])
    global html1, html2, t1, t2, t1w, t2w, t1t, t2t, kw1, desc1, kw2, desc2, title1, title2
    t1 = []
    t2 = []
    response =  urllib.request.urlopen(url1)
    html1 = response.read()
    response =  urllib.request.urlopen(url2)
    html2 = response.read()
    text1, title1, kw1, desc1 = text_from_html(html1)
    t1 = Lemmatise(text1)
    t1t = Lemmatise(str(title1))
    text2, title2, kw2, desc2 = text_from_html(html2)
    t2 = Lemmatise(text2)
    t2t = Lemmatise(str(title2))

    # Make a list of words and their counts. We will take the top 10 for further analysis
    CountWords(t1)
    t1w = []
    for key,val in counts.items():
        item = str(val) + ':' + str(key.lower())
        t1w.append(item)
    t1w.sort(key=lambda fname: int(fname.split(':')[0]), reverse=True)
    CountWords(t2)
    t2w = []
    for key,val in counts.items():
        item = str(val) + ':' + str(key.lower())
        t2w.append(item)
    t2w.sort(key=lambda fname: int(fname.split(':')[0]), reverse=True)
    
def GetDocSegments():
    # Concatenate 100 words each from t1 and t2 into separate strings and add to two arrays, 'doc1_segs' and 'doc2_segs'
    global doc1_segs, doc2_segs
    # Split and save the text as 100 word segments
    doc1_segs = []
    doc2_segs = []
    text = ''
    # Split t1 and t2 into text of 100 words
    l = len(t1)
    n = int(l/100)
    for i in range(n):
        b = i*100
        e = b+100
        for j in range(b,e):
            text += t1[j] + ' '
        doc1_segs.append(text)
        text = ''

    l = len(t2)
    n = int(l/100)
    for i in range(n):
        b = i*100
        e = b+100
        for j in range(b,e):
            text += t2[j] + ' '
        doc2_segs.append(text)
        text = ''

    l = len(doc1_segs)    
    m = len(doc2_segs) 
    return l,m

def CalculateSimilarity(i,j,nurls):
    l,m = GetDocSegments() # Split the doc into small chunks
    sim = 0.00
    sims = []
    tot = 0.00
    k = nurls
    if (l < k):
        k = l
    if (m < k):
        k = m
    if(k == 0):
        return 0
    for i in range(k):
        for j in range(k):
            doc1 = nlp(doc1_segs[i])
            doc2 = nlp(doc2_segs[j])
            sim = doc1.similarity(doc2)
            sims.append(sim)
    sims.sort(reverse=True)
    for i in range(k):
        tot += sims[i]
    avg = round(tot/k,2)
    return avg

def CalculateWordFrequency():
    tw1f = []
    tw2f = []
    nw = 10 # Number of words in the list to be used
# ------------t1--------------
    lw = len(t1w)
    tot = 0
    # Count the total number of words
    for i in range (0,nw):
        fields = t1w[i].split(":")
        tot += int(fields[0])
    # Take the percentage of top 10 words in t1w
    for i in range (0,nw):
        fields = t1w[i].split(":")
        c = int(fields[0])
        w = fields[1]
        f = str(round(float(c/tot*100),2))
        fw = f + ":" + w
        tw1f.append(fw)
# ------------t2--------------
    lw = len(t2w)
    tot = 0
    # Count the total number of words
    for i in range (0,nw):
        fields = t2w[i].split(":")
        tot += int(fields[0])
    for i in range (0,nw):
        fields = t2w[i].split(":")
        c = int(fields[0])
        w = fields[1]
        f = str(round(float(c/tot*100),2))
        fw = f + ":" + w
        tw2f.append(w)
# Compare the two to get the word frequency
    lw1 = len(tw1f)
    lw2 = len(tw2f)
    mf = 0.00
    for i in range(lw2):
        w2 = tw2f[i]
        tf = 0.00
        for j in range(lw1):
            fields = tw1f[j].split(":")
            f1 = float(fields[0])
            w1 = fields[1]
            tf += f1
            if(w1 == w2):
                mf += f1
    freq = round(mf/tf,2)
    return freq

def Title_Match(t1t, t2t):
    tv = 0.00
    for w1 in (t1t):
        for w2 in (t2t):
            if (w1 == w2):
                tv = 0.1
    return tv

def Domain_match(url1, url2):
    dv = 0.00
    f1 = url1.split('/')
    d1 = f1[2].replace('www.','')
    f2 = url2.split('/')
    d2 = f2[2].replace('www.','')
    if(d1 == d2):
        dv = 1.0
    return dv

# Only the visible text is to be retrieved
def tag_visible(element):
    if element.parent.name in ['a', 'link', 'style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    keywords = ''
    description = ''
    title = soup.title.string
    for tag in soup.find_all("meta"):
        if tag.get("name", None) == "og:title":
            title = tag.get("content", None)
            pass
        if tag.get("name", None) == "keywords":
            keywords = tag.get("content", None)
            pass
    
        if tag.get("name", None) == "description":
            description = tag.get("content", None)
            pass
    
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts), title, keywords, description

def SimilarityScore(t1, t2):
    doc1 = nlp(t1)
    doc2 = nlp(t2)
    sim = round(doc1.similarity(doc2),2)
    return sim


In [8]:
# Topics: Variable
urls = [
    "https://www.ands.org.au/working-with-data/metadata/geospatial-data-and-metadata",
    "https://www.ands.org.au/guides/geospatial",
    "https://www.safe.com/what-is/spatial-data/",
    "https://gisgeography.com/what-is-geodata-geospatial-data/",
    "https://www.mathworks.com/help/map/what-is-geospatial-data.html",
    "https://www.veris.com.au/our-services/geospatial-data-management/",

    "https://searchbusinessanalytics.techtarget.com/definition/natural-language-processing-NLP",
    "https://algorithmia.com/blog/introduction-natural-language-processing-nlp",
    "https://www.coursera.org/learn/language-processing",
    "https://en.wikipedia.org/wiki/Natural_language_processing",
    "https://www.forbes.com/sites/bernardmarr/2019/06/03/5-amazing-examples-of-natural-language-processing-nlp-in-practice/",
    "https://www.sas.com/en_au/insights/analytics/what-is-natural-language-processing-nlp.html",

    "https://aiatsis.gov.au/explore/articles/indigenous-australian-languages",
    "https://en.wikipedia.org/wiki/Australian_Aboriginal_languages",
    "https://www.commonground.org.au/learn/indigenous-languages-avoiding-a-silent-future",
    "http://theconversation.com/the-state-of-australias-indigenous-languages-and-how-we-can-help-people-speak-them-more-often-109662",
    "https://www.clc.org.au/articles/info/aboriginal-languages/",
    "https://www.britannica.com/topic/Australian-Aboriginal-languages",   

    "https://en.wikipedia.org/wiki/Human_genome",
    "https://en.wikipedia.org/wiki/Human_Genome_Project",
    "https://www.nature.com/scitable/topicpage/dna-sequencing-technologies-key-to-the-human-828/",
    "https://www.britannica.com/event/Human-Genome-Project/Advances-based-on-the-HGP",
    "https://www.genome.gov/human-genome-project/Completion-FAQ",
    "https://www.yourgenome.org/stories/how-is-the-completed-human-genome-sequence-being-used",   

    "https://en.wikipedia.org/wiki/Machine_learning",
    "https://www.sas.com/en_au/insights/analytics/machine-learning.html", 
    "https://en.wikipedia.org/wiki/London", 
    "https://en.wikipedia.org/wiki/England",
    "https://en.wikipedia.org/wiki/Titanic_(1997_film)", 
    "https://en.wikipedia.org/wiki/Titanic_(1953_film)", 
]
nurls = len(urls)

In [21]:
#-----------------------------------------------------------------------------------------------------------------
# Call the functions
# Read the HTML contents and lemmatise into two arrays, 't1' and 't2'
i = 0
j = 2
ReadUrls(i, j)
# Calculate the cosine similarity between text, titles, key words and descriptions
# Some of these will be 0. Not all need to be used.
sc = CalculateSimilarity(i,j,nurls)
st = SimilarityScore(str(title1),str(title2))
sk = SimilarityScore(kw1,kw2)
sd = SimilarityScore(desc1,desc2)
wf = CalculateWordFrequency()
dv = Domain_match(urls[i], urls[j])

print('sc:', sc , '|', 'st:', st, '|', 'sk:', sk, '|', 'sd:', sd, '|', 'wf:', wf, '|', 'dv:', dv, '\n')


sc: 0.95 | st: 0.75 | sk: 0.0 | sd: 0.96 | wf: 0.36 | dv: 0.0 



In [ ]:
h = "#\t"
o = ""
for i in range (nurls):
    h += str(i) + '\t'
#h += '\n'  
print(h)
#print('Blah')
def AddTabs(i):
    global o
    for j in range(i):
        o += '\t'
for i in range (nurls):
    o = str(i)
    AddTabs(i+1)
    for j in range (i,nurls):
        ReadUrls(i, j)
        st = SimilarityScore(str(title1),str(title2))
        sk = SimilarityScore(kw1,kw2)
        sd = SimilarityScore(desc1,desc2)
        dv = Domain_match(urls[i], urls[j])
        GetDocSegments()
        sc = CalculateSimilarity(i,j,nurls)
        wf = CalculateWordFrequency()
#        s = round((sc + wf + tv + dv),2)
#        s = round((sc + wf + dv + st + sk + sd),2)
#        s = round(sc,2)
        s = round((sc+wf),2)
        o += str(s) + '\t'
    print(o)


#	0	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	
0	2.0	1.64	1.31	1.24	1.41	1.37	1.3	0.91	0.9	0.9	0.89	1.13	0.87	0.83	0.77	0.8	0.77	0.94	0.83	0.94	1.1	0.88	0.89	0.89	1.14	1.13	0.83	0.78	0.77	0.71	
1		2.0	1.12	1.11	1.38	1.25	1.25	0.88	0.89	0.88	0.84	0.47	0.87	0.82	0.71	0.78	0.7	1.13	0.76	0.93	0.87	0.89	0.86	0.82	0.96	0.47	0.71	0.71	0.72	0.74	


# __________End of page__________